# Apply Ranking Recovery Algorithms

In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
# create a FairPairGraph with disadvantaged minority
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
# run 100 iterations of ProbKnockout sampling
sampler = OversampleMinority(G, warn=False)
sampler.apply(iter=1000, k=1)
# apply davidScore for ranking recovery
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method
ranker._print_with_score(ranking) # sorted by rank score

,node,orig score,rank score
0,2,0.953540,16.872006
1,3,0.999900,12.384279
2,7,0.840066,11.133229
3,4,0.726021,9.904065
4,5,0.742329,9.210471
5,13,0.363570,7.479284
6,1,0.502196,5.479048
7,6,0.254406,2.668210
8,8,0.287535,2.666425
9,11,0.333772,2.371359


## Metrics

In [4]:
# score-based metrics
print("Majority NDCG (by score):", NDCG(G.majority, ranking)) # NDCG is only calculated as a "in-group" comparison because it is normalized
print("Minority NDCG (by score):", NDCG(G.minority, ranking))
print("Majority MSE (by score):", MSE(G.majority, ranking))
print("Minority MSE (by score):", MSE(G.minority, ranking))

Majority NDCG (by score): 0.9911708923483539
Minority NDCG (by score): 0.9999999999999999
Majority MSE (by score): 0.09999502061195491
Minority MSE (by score): 0.17813764655060174


In [5]:
# rank-based metrics
print("Majority mean error (by rank):", rank_mean_error(G, ranking, G.majority))
print("Minority mean error (by rank):", rank_mean_error(G, ranking, G.minority))
print("Majority MSE (by rank):", rank_MSE(G, ranking, G.majority))
print("Minority MSE (by rank):", rank_MSE(G, ranking, G.minority))
print("Majority Spearman's rho:", spearmanr(G, ranking, G.majority))
print("Minority Spearman's rho:", spearmanr(G, ranking, G.minority))

Majority mean error (by rank): -0.3
Minority mean error (by rank): 0.6
Majority MSE (by rank): 7.5
Minority MSE (by rank): 1.8
Majority Spearman's rho: PearsonRResult(statistic=0.7319340236667495, pvalue=0.016104897630961866)
Minority Spearman's rho: PearsonRResult(statistic=0.9592822086170635, pvalue=0.009802565122580105)


In [6]:
# weighted Kendall tau
print("Majority weighted Kendall tau:", weighted_tau(G, ranking, G.majority))
print("Minority weighted Kendall tau:", weighted_tau(G, ranking, G.minority))

Majority weighted Kendall tau: 0.016004196657975166
Minority weighted Kendall tau: 0.04731395387384422
